In [1]:
library(RODBC)
library(DBI)
library(HelpersMG)
library(stringr)
library(dplyr)
library(ggplot2)
library(lattice)
library(mechkar)
library(matrixStats)

setwd("C://bb//airbnb")

Loading required package: lme4

Loading required package: Matrix

Loading required package: coda


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'matrixStats'


The following object is masked from 'package:dplyr':

    count




## Select FF data from database


In [2]:


#con = odbcConnect("airbnb")
#airbnb_ff<-sqlQuery(con, "select * from dbo.FF_V")
#save(airbnb_ff, file="C://bb//insideairbnb//airbnb_ff.RData")

load("C://bb//insideairbnb//airbnb_ff.RData")    

dim(airbnb_ff)

df<-airbnb_ff
df<-as.data.frame(df)

head(df)

[1] 747184     65

,listing_id,week_num,year_num,cal_period,avg_price,min_price,max_price,occupied,occur,rownames,...,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,period,calendar_updated_trim,host_seniority
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,...,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<fct>,<int>
1,27267054,12,2019,10,139,139,139,7,7,76304,...,10,t,flexible,f,f,1,5.78,10,today,4
2,27267054,13,2019,10,139,139,139,1,7,76304,...,10,t,flexible,f,f,1,5.78,10,today,4
3,27267054,11,2019,10,139,139,139,4,7,76304,...,10,t,flexible,f,f,1,5.78,10,today,4
4,27267054,14,2019,10,139,139,139,5,7,76304,...,10,t,flexible,f,f,1,5.78,10,today,4
5,13762975,12,2019,10,85,85,85,7,7,73498,...,9,f,moderate,f,f,1,2.59,10,today,7
6,13762975,11,2019,10,85,85,85,7,7,73498,...,9,f,moderate,f,f,1,2.59,10,today,7


## Check data values

In [3]:
summary(df)

   listing_id          week_num        year_num      cal_period   
 Min.   :     958   Min.   : 1.00   Min.   :2015   Min.   : 1.00  
 1st Qu.: 4277696   1st Qu.:16.00   1st Qu.:2016   1st Qu.:11.00  
 Median :10789800   Median :30.00   Median :2017   Median :25.00  
 Mean   :12504705   Mean   :29.01   Mean   :2017   Mean   :25.47  
 3rd Qu.:18738629   3rd Qu.:43.00   3rd Qu.:2019   3rd Qu.:39.00  
 Max.   :40569280   Max.   :53.00   Max.   :2020   Max.   :50.00  
                                                                  
   avg_price           min_price           max_price           occupied    
 Min.   :     0.00   Min.   :     0.00   Min.   :     0.0   Min.   :0.000  
 1st Qu.:     0.00   1st Qu.:     0.00   1st Qu.:     0.0   1st Qu.:3.000  
 Median :    59.57   Median :     0.00   Median :   110.0   Median :6.000  
 Mean   :   111.05   Mean   :    80.11   Mean   :   154.5   Mean   :4.773  
 3rd Qu.:   146.00   3rd Qu.:   117.00   3rd Qu.:   199.0   3rd Qu.:7.000  
 Max.   

## Remove garbage entries
### avg_price == 0
### avg_proce == 0
### beds == 0



In [4]:
df.clean<-df %>% filter(avg_price > 0.00 & min_price > 0.00 & beds > 0)

In [5]:
dim(df)
dim(df.clean)
df<-df.clean

[1] 747184     65

[1] 283544     65

## Set Y value (property occupation) accroding to the following rules:
## 0 - 2 -> "Low"
## 3 - 6 -> "Medium"
## 7      -> "Full"

In [6]:
df <- df %>% mutate(occupied_cat = factor(ifelse(occupied >= 7, "Full", ifelse(occupied<3, "Low", "Medium"))))

In [7]:
dim(df %>% filter(occupied_cat == "Full"))


[1] 97952    66

In [8]:
dim(df %>% filter(occupied_cat == "Medium"))

[1] 54837    66

In [9]:
dim(df %>% filter(occupied_cat == "Low"))

[1] 130755     66

In [10]:
tbl <- table(df$occupied_cat)
cbind(tbl,prop.table(tbl) * 100)


,tbl,
Full,97952,34.54561
Low,130755,46.11454
Medium,54837,19.33986


## Remove technical/redundant/highly missing fields
### calendar_updated_trim - was used for data retrieval

### period - was used for data retrieval', 'calendar_updated', 'rownames', 'occur', 'id',
               'last_scraped', 'host_since', 'cal_period', 'square_feet', 'year_num', 
               "weekly_price", "monthly_price", "occupied")
### host_total_listings_count - redundant (calculated_host_listings_count)
### host_listings_count - redundant (calculated_host_listings_count)
### occupied - this value is categorized and replaced by occupied_cat
### review_scores_rating - this value is calculated as weighted sum of other scores



In [11]:
tech_fileds<-c('calendar_updated_trim', 'period', 'calendar_updated', 'rownames', 'occur', 'id', 'price',
               'last_scraped', 'cal_period', 'square_feet', 'year_num', "weekly_price", "monthly_price", 
               "host_since", "host_total_listings_count", "occupied", "host_listings_count", "review_scores_rating")
df<-df[,setdiff(names(df), tech_fileds)]

## Store in FF.csv

In [12]:
write.csv(df,'./data/FF.csv')